In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fingerprint import GraphFingerprint
from wb import WeightsAndBiases
from itertools import combinations
from random import choice, sample
from numpy.random import permutation
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split, ShuffleSplit, cross_val_score
from sklearn.preprocessing import LabelBinarizer
from autograd import grad
from time import time

import autograd.numpy as np
import networkx as nx
import math
import matplotlib.pyplot as plt
from numba import jit

In [3]:
shapes = dict()
shapes[0] = 10
shapes[1] = 10
shapes[2] = 10
wb = WeightsAndBiases(2, shapes)
# wb[0]

In [9]:
def make_random_graph(nodes, n_edges, features_dict):
    """
    Makes a randomly connected graph. 
    """
    
    G = nx.Graph()
    for n in nodes:
        G.add_node(n, features=features_dict[n])
    
    for i in range(n_edges):
        u, v = sample(G.nodes(), 2)
        G.add_edge(u, v)
        
    return G

# features_dict will look like this:
# {0: array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
#  1: array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0]),
#  2: array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
#  3: array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
#  4: array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
#  5: array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0]),
#  6: array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0]),
#  7: array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0]),
#  8: array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
#  9: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])}

all_nodes = [i for i in range(10)]    
lb = LabelBinarizer()
features_dict = {i:lb.fit_transform(all_nodes)[i] for i in all_nodes}

G = make_random_graph(sample(all_nodes, 6), 5, features_dict)
G.edges(data=True)
# G.nodes(data=True)

[(0, 5, {}), (4, 8, {}), (4, 5, {}), (5, 8, {})]

In [10]:
def score(G):
    """
    The regressable score for each graph will be the sum of the 
    (square root of each node + the sum of its neighbors.)
    """
    sum_score = 0
    for n, d in G.nodes(data=True):
        sum_score += math.sqrt(n)
        
        for nbr in G.neighbors(n):
            sum_score += nbr
    return sum_score

score(G)

49.21075947218795

In [11]:
syngraphs = [make_random_graph(sample(all_nodes, 6), 5, features_dict) for i in range(1000)]

In [12]:
len(syngraphs)

1000

Set up a learning scenario where...

In [13]:
fingerprints = np.zeros((len(syngraphs), 10))

for i, g in enumerate(syngraphs):
    gfp = GraphFingerprint(g, 2, shapes)
    fp = gfp.compute_fingerprint(wb.vect, wb.unflattener)
    fingerprints[i] = fp

In [15]:
import pandas as pd
X = pd.DataFrame(np.array(fingerprints))
Y = [score(g) for g in syngraphs]
Y

[48.928198407402256,
 74.20622924337965,
 54.33788174096706,
 47.15973615609376,
 57.74552259372066,
 48.099771282656064,
 56.47870866461908,
 38.24150542378974,
 56.29603285093747,
 50.00996767509825,
 52.88839199818388,
 54.21075947218795,
 57.51398484502916,
 37.62044280575275,
 44.29603285093748,
 22.382332347441764,
 49.57394971846685,
 34.74552259372066,
 71.47870866461908,
 37.44229722087945,
 35.83182209022495,
 43.21075947218795,
 48.79201568100656,
 43.06335983891644,
 46.685557720282965,
 70.15973615609374,
 50.337881740967056,
 57.88839199818388,
 54.974691494688166,
 60.331309031347566,
 33.028083658506354,
 48.33130903134756,
 49.028083658506354,
 56.24603565259803,
 50.33130903134756,
 67.33788174096706,
 75.57394971846685,
 73.62044280575276,
 48.69213042990247,
 42.57394971846685,
 61.928198407402256,
 51.61387009613326,
 52.21075947218795,
 70.06993254853595,
 55.12445997568367,
 47.24150542378975,
 68.92819840740225,
 59.18154055035206,
 46.47870866461908,
 46.210759

In [16]:
# A simple test - the weights are random, so given the random weights, what is the prediction accuracy using
# random forest?

cv = ShuffleSplit(n=len(X), n_iter=10)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

In [17]:
rfr = RandomForestRegressor()
rfr.fit(X_train, Y_train)
# preds = np.rint(rfr.predict(X_test))
preds = rfr.predict(X_test)

from sklearn.metrics import mean_squared_error as mse

print(preds)
mse(preds, Y_test)

[ 43.67695261  47.3385884   57.87213266  58.46154494  49.30283591
  45.1910689   49.62011753  68.51271188  55.97949475  56.87368079
  51.20740201  52.82036204  39.96014671  45.98629516  38.3348929
  47.1899939   51.29653208  55.64410408  68.73036695  39.97548537
  52.03298246  41.89859511  54.11502223  66.73106219  42.47465259
  56.41343114  50.86075826  53.47209566  48.60856569  36.44817545
  55.41267125  41.57927364  58.88869173  62.32523781  63.10703161
  75.18985164  52.90939361  55.56826118  62.29001719  55.95944492
  56.5585834   42.65502314  62.62820979  40.00982918  37.80423368
  57.50874823  46.29515216  55.92392279  54.15469834  39.64106818
  53.82318679  46.13854437  52.59131282  60.15372546  48.14311435
  56.10719572  49.29198149  55.15306164  66.20213352  57.56156667
  66.99253059  44.18506805  56.10373662  49.92821683  54.3671003
  53.42356048  53.99011181  56.83291065  46.76118799  36.61827233
  42.63446111  66.27867992  39.85271934  60.13191935  38.0692856
  54.57874676

97.212967823908713

In [18]:
# How does this compare with randomly shuffled data?
mse(permutation(Y_test), Y_test)

255.20357633857947

In [19]:
[i for i in zip(Y_test, preds)]

[(44.15973615609376, 43.676952614980749),
 (51.92819840740225, 47.338588399393792),
 (69.15973615609374, 57.872132655583229),
 (61.51398484502916, 58.461544944567756),
 (49.242640687119284, 49.302835909230751),
 (53.24603565259804, 45.191068904758133),
 (22.831822090224943, 49.620117530665411),
 (72.92366817859397, 68.512711883478048),
 (72.89178696366264, 55.979494751521848),
 (45.12445997568367, 56.873680790125398),
 (54.29603285093748, 51.20740201163337),
 (36.29603285093748, 52.82036203784115),
 (37.099771282656064, 39.960146714030557),
 (35.41760852785184, 45.986295155139111),
 (26.124459975683667, 38.334892896194745),
 (38.009967675098245, 47.189993902282637),
 (62.928198407402256, 51.296532083596801),
 (53.06335983891644, 55.644104084057894),
 (46.028083658506354, 68.730366945905601),
 (52.33788174096706, 39.97548537008025),
 (60.331309031347566, 52.032982460519904),
 (39.33130903134756, 41.898595111413329),
 (43.337881740967056, 54.11502223219086),
 (68.06335983891644, 66.73106

# Optimization with Autograd

Here, I try using autograd to do the optimizations required.

In [20]:
def predict(wb_vect, wb_unflattener, graph_fp):#, linweights):
    """
    Given the weights and biases for each layer, make a prediction for the graph.
    """
    fp = graph_fp.compute_fingerprint(wb_vect, wb_unflattener)
    wb = wb_unflattener(wb_vect)
    top_layer = max(wb.keys())
    linweights = wb[top_layer]['linweights']
    return np.dot(fp, linweights)

predict(wb.vect, wb.unflattener, gfp)

array([[ 0.47141931]])

In [21]:
@jit
def train_loss(wb_vect, wb_unflattener):
    """
    Training loss function - should take in a vector.
    """
    sum_loss = 0
    for i, g in enumerate(syngraphs):
        gfp = GraphFingerprint(g, 2, shapes)
        pred = predict(wb_vect, wb_unflattener, gfp)
        loss = len(g.nodes()) - predict(wb_vect, wb_unflattener, gfp)
        sum_loss = sum_loss + loss ** 2
    
    return sum_loss / len(syngraphs)

train_loss(wb.vect, wb.unflattener)

array([[ 30.52793686]])

In [22]:
def sgd(grad, wb_vect, wb_unflattener, callback=None, num_iters=200, step_size=0.1, mass=0.9):
    """
    Stochastic gradient descent with momentum.
    """
    velocity = np.zeros(len(wb_vect))
    for i in range(num_iters):
        print(i)
        g = grad(wb_vect, wb_unflattener)
        # if callback: callback(x, i, g)
        velocity = mass * velocity - (1.0 - mass) * g
        wb_vect += step_size * velocity
        print(train_loss(wb_vect, wb_unflattener))
    return wb_vect

In [23]:
train_loss(wb.vect, wb.unflattener)

array([[ 30.52793686]])

In [24]:
grad_func = grad(train_loss)

In [36]:
sgd(grad_func, wb.vect, wb.unflattener, num_iters=200)

0
[[ 1.17223147]]
1
[[ 1.08935819]]
2
[[ 0.9782466]]
3
[[ 0.84884053]]
4
[[ 0.71082609]]
5
[[ 0.57295617]]
6
[[ 0.44257404]]
7
[[ 0.32533199]]
8
[[ 0.22508747]]
9
[[ 0.1439499]]
10
[[ 0.08244543]]
11
[[ 0.03976491]]
12
[[ 0.01406122]]
13
[[ 0.00276477]]
14
[[ 0.00289126]]
15
[[ 0.01132097]]
16
[[ 0.02503518]]
17
[[ 0.0413011]]
18
[[ 0.05780206]]
19
[[ 0.0727142]]
20


KeyboardInterrupt: 

In [37]:
trained_weights = wb.unflattener(wb.vect)[2]['linweights']
trained_weights

array([[ 1.08165481],
       [ 1.14253854],
       [ 5.90582944],
       [ 1.41045033],
       [ 1.4167689 ],
       [ 1.04994567],
       [ 0.96421758],
       [ 1.90682434],
       [ 1.67905653],
       [ 0.95807283]])

In [38]:
test_graphs = [make_random_graph(sample(all_nodes, 6), 5, features_dict) for i in range(100)]

test_fingerprints = np.zeros((len(test_graphs), 10))
# test_fingerprints
for i, g in enumerate(test_graphs):
    gfp = GraphFingerprint(g, 2, shapes)
    fp = gfp.compute_fingerprint(wb.vect, wb.unflattener)
    test_fingerprints[i] = fp

# test_fingerprints

TypeError: super(type, obj): obj must be an instance or subtype of type

In [35]:
preds = []
for i, g in enumerate(test_graphs):
    gfp = GraphFingerprint(g, 2, shapes)
#     fp = gfp.compute_fingerprint(wb.vect, wb.unflattener)
    preds.append(predict(wb.vect, wb.unflattener, gfp)[0])
# preds[0]

In [33]:
Y_test = [score(g) for g in syngraphs]

[i for i in zip(Y_test, preds)]

[(48.928198407402256, array([ 7.14867291])),
 (74.20622924337965, array([ 7.09265279])),
 (54.33788174096706, array([ 7.13102324])),
 (47.15973615609376, array([ 7.14004252])),
 (57.74552259372066, array([ 7.15536322])),
 (48.099771282656064, array([ 7.08721323])),
 (56.47870866461908, array([ 7.09514357])),
 (38.24150542378974, array([ 7.14967854])),
 (56.29603285093747, array([ 7.14834682])),
 (50.00996767509825, array([ 7.03731036])),
 (52.88839199818388, array([ 7.11407215])),
 (54.21075947218795, array([ 7.00740655])),
 (57.51398484502916, array([ 7.1114426])),
 (37.62044280575275, array([ 7.15784038])),
 (44.29603285093748, array([ 7.10304284])),
 (22.382332347441764, array([ 7.158675])),
 (49.57394971846685, array([ 7.14029349])),
 (34.74552259372066, array([ 7.13034657])),
 (71.47870866461908, array([ 7.03841688])),
 (37.44229722087945, array([ 7.10595887])),
 (35.83182209022495, array([ 7.09429304])),
 (43.21075947218795, array([ 7.12786603])),
 (48.79201568100656, array([ 7.0

In [ ]:
plt.scatter(preds, n_nodes, alpha=0.3)
plt.xlabel('predictions')
plt.ylabel('actual')
plt.title('number of nodes')

In [ ]:
class Class(object):
    """docstring for ClassName"""
    def __init__(self, arg):
        super(Class, self).__init__()
        self.arg = arg
        
    def __iter__():
        pass
        
    def function(self, value, other_thing):
        return value['k']['v']['x'] ** 2 + value['y'] ** 3
    
    def function2(self, value):
        return np.sum(np.dot(value['arr1'], value['arr2'])) + 1
        
        
# def function(value):
#     return value ** 2

In [ ]:
c = Class(np.random.random((10,10)))

from collections import OrderedDict
value = dict({'k':{'v':{'x':3.0}}, 'y':2.0})
gradfunc = grad(c.function)
gradfunc(value, 'string')

In [ ]:
def fun2(value):
    return np.sum(np.dot(value['arr1'], value['arr2']))

value = {'arr1':np.random.random((10,10)), 'arr2':np.random.random((10,10))}
gradfunc = grad(fun2)(value)
gradfunc

In [ ]:
value = {'arr1':np.random.random((10,10)), 'arr2':np.random.random((10,10))}
# value
gradfunc = grad(c.function2)
gradfunc(value)
# np.dot(c.arg, value['arr1'])# , c.arg)
# c.function2(value)

In [ ]:
np.dot(value['arr1'], value['arr2'])